In [1]:
import lancedb
import pyarrow as pa
import pandas as pd
from datasets import load_dataset
from transformers import GPT2Tokenizer

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Para este ejercicio descargamos una base de datos de mensajes de texto en inglés.

In [2]:
dataset = load_dataset("chirunder/text_messages")

In [3]:
df = pd.DataFrame(dataset['train'])
df.rename(columns={'text': 'texto'}, inplace=True)
df = df.head(500)

Para los siguientes ejercicios, voy a crear una variable del numero de palabras en cada mensaje de texto.

In [4]:
df['n'] = df['texto'].apply(lambda x: len(str(x).split()))
df = df[['n', 'texto']]
df.head()

,n,texto
0,13,Top right I gained a little speed with the add...
1,14,They are heavier wheels though as are all the ...
2,9,Federally registering a trademark is more than...
3,21,I'll have to jog my memory from rooting a few ...
4,10,Unless you can afford to buy all new larger cl...


## Task 1:
A partir del dataframe df, crea df_tokenized (usando el Tokenizer de GPT2) con dos columnas pero con el texto tokenizado. Asegurate de que todos los embeddings tengan la misma longitud y los tokens sean enteros (todos enteros o todos doubles). 

In [17]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
texto_tokenized = df['texto'].apply(lambda x: tokenizer(x)["input_ids"])
texto_tokenized = texto_tokenized.apply(lambda x: x[:300] + [0] * (300 - len(x)) if len(x) < 300 else x[:300])

df_tokenized = pd.DataFrame({'vector': texto_tokenized, 'length': df['n']})
print(df_tokenized)

/usr/local/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


                                                vector  length
0    [9126, 826, 314, 8618, 257, 1310, 2866, 351, 2...      13
1    [2990, 389, 20140, 13666, 996, 355, 389, 477, ...      14
2    [37, 5702, 453, 28336, 257, 16028, 318, 517, 6...       9
3    [40, 1183, 423, 284, 48342, 616, 4088, 422, 40...      21
4    [28042, 345, 460, 5368, 284, 2822, 477, 649, 4...      10
..                                                 ...     ...
495  [5703, 20536, 329, 345, 0, 0, 0, 0, 0, 0, 0, 0...       4
496  [40, 481, 307, 3772, 618, 262, 31143, 389, 257...      12
497  [71, 12236, 262, 21296, 286, 502, 1719, 3621, ...       9
498  [1026, 318, 1016, 262, 835, 286, 2011, 14106, ...      11
499  [40, 1101, 14442, 262, 3326, 12894, 2442, 319,...      23

[500 rows x 2 columns]


## Task 2:
Mete el dataframe a una tabla en una base de datos de LanceDB.

In [19]:
db = lancedb.connect("./.lancedb")

if "tabla" in db.table_names():
    db.drop_table("tabla")

db.create_table("tabla", df_tokenized)
db["tabla"].head()

pyarrow.Table
vector: fixed_size_list<item: float>[300]
  child 0, item: float
length: int64
----
vector: [[[9126,826,314,8618,257,...,0,0,0,0,0],[2990,389,20140,13666,996,...,0,0,0,0,0],[37,5702,453,28336,257,...,0,0,0,0,0],[40,1183,423,284,48342,...,0,0,0,0,0],[28042,345,460,5368,284,...,0,0,0,0,0]]]
length: [[13,14,9,21,10]]

## Task 3:
Haz una query estilo SQL a la tabla de la base de datos. Quiero que escribas la query equivalente y pongas la explicación de lo que está haciendo la consulta. Hint: usa la columna "n". 

In [23]:
(db["tabla"].search()
    .where("length < 4")
    .select(["vector", "length"])
    .limit(11)
    .to_pandas())

,vector,length
0,"[5195.0, 41320.0, 2767.0, 30.0, 0.0, 0.0, 0.0,...",2
1,"[40732.0, 30.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
2,"[53.0, 425.0, 4689.0, 4881.0, 0.0, 0.0, 0.0, 0...",3
3,"[7376.0, 364.0, 8518.0, 13.0, 0.0, 0.0, 0.0, 0...",2
4,"[2396.0, 340.0, 2540.0, 13.0, 0.0, 0.0, 0.0, 0...",3
5,"[43.0, 76.0, 69.0, 5488.0, 1049.0, 3404.0, 0.0...",3
6,"[14108.0, 5170.0, 502.0, 0.0, 0.0, 0.0, 0.0, 0...",3
7,"[17821.0, 393.0, 407.0, 13.0, 0.0, 0.0, 0.0, 0...",3
8,"[34.0, 5691.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
9,"[10814.0, 15744.0, 260.0, 13.0, 0.0, 0.0, 0.0,...",2


- Query en SQL equivalente:

SELECT vector, length
FROM tabla
WHERE length < 4
LIMIT 11;

- Explicacion: Buscamos los mensajes con una longitud menor a 4 palabras

## Task 4:
Inventa un mensaje de texto que tu podrías escribirle a un amigo. Tokenizalo y ponlo en el formato adecuado para hacer un vector query. Quiero que me regreses el mensaje más parecido al mensaje que inventaste (OJO: quiero el texto, no el embedding). HINT: Hay que decodear el resultado del query.

In [33]:
mensaje_tokenized = tokenizer("My name is Jeff")["input_ids"]
print(mensaje_tokenized)

n = 300
def ajustar_vector(input, n):
    output = input[:n]
    
    while len(output) < n:
        output.append(0)
    
    return output
    
mensaje_tokenized = ajustar_vector(mensaje_tokenized, n)
print(mensaje_tokenized)

[3666, 1438, 318, 5502]
[3666, 1438, 318, 5502, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [34]:
resultados = (db["tabla"].search(mensaje_tokenized)
    .metric("cosine")
    .select(["vector", "length"])
    .limit(11)
    .to_pandas())

primer_vector = resultados['vector'][0]
primer_vector_sin_ceros = [token for token in primer_vector if token != 0]
texto_decodificado = tokenizer.decode(primer_vector_sin_ceros, skip_special_tokens=True)
print(texto_decodificado)

Just put some Thanks.!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
